**Libraries**

In [1]:
import pandas as pd
import librosa
import tensorflow as tf
import numpy as np
from tqdm import tqdm
import pyaudio
import wave

**Reading the CSV File.**

In [2]:
df = pd.read_csv('cv-valid-train.csv')
df = df[~df['gender'].isna()]

**Train Test Split**

In [3]:
BASE_DIR = 'cv-valid-train/'
X_train, y_train = [], []

**Defining Audio features**

In [4]:
for i in tqdm(df.index):
    file = BASE_DIR + df.loc[i, 'filename']
    wf, sr = librosa.load(file)
    mfcc_wf = librosa.feature.mfcc(y=wf, sr=sr)
    b = tf.keras.utils.pad_sequences(mfcc_wf, padding='post', maxlen=200)
    X_train.append(b)
    if df.loc[i, 'gender'] == 'male':
        y_train.append(1)
    else:
        y_train.append(0)

100%|██████████| 74059/74059 [17:40<00:00, 69.82it/s]  


In [5]:
X_train = np.array(X_train)
y_train = np.array(y_train)

**DataFrame**

In [6]:
df

,filename,text,up_votes,down_votes,age,gender,accent,duration
5,cv-valid-train/sample-000005.mp3,a shepherd may like to travel but he should ne...,1,0,twenties,female,us,NaN
8,cv-valid-train/sample-000008.mp3,put jackie right on the staff,3,0,seventies,male,us,NaN
13,cv-valid-train/sample-000013.mp3,but he had found a guide and didn't want to mi...,1,0,thirties,female,us,NaN
14,cv-valid-train/sample-000014.mp3,as they began to decorate the hallway a silhou...,1,0,sixties,male,england,NaN
19,cv-valid-train/sample-000019.mp3,then they got ahold of some dough and went goofy,1,0,fifties,male,australia,NaN
...,...,...,...,...,...,...,...,...
195766,cv-valid-train/sample-195766.mp3,but before i go i want to tell you a little story,1,0,fourties,male,england,NaN
195767,cv-valid-train/sample-195767.mp3,down below in the darkness were hundreds of pe...,3,0,thirties,female,us,NaN
195770,cv-valid-train/sample-195770.mp3,he heard a muffled grating sound and saw the b...,4,0,twenties,male,england,NaN
195771,cv-valid-train/sample-195771.mp3,the englishman said nothing,1,0,thirties,male,england,NaN


**Building The Model**

In [7]:
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(20, 200,)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu', kernel_regularizer=tf.keras.regularizers.l2()),
    tf.keras.layers.Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2()),
    tf.keras.layers.Dense(64, activation='relu', kernel_regularizer=tf.keras.regularizers.l2()),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

**Compiling Model**

In [8]:
model.compile(optimizer='adam', loss=tf.keras.losses.BinaryCrossentropy(), metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=100, validation_split=0.2)
train_accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']


Epoch 1/100


1852/1852 [==============================] - 21s 11ms/step - loss: 2.7108 - accuracy: 0.8268 - val_loss: 1.1954 - val_accuracy: 0.8249
Epoch 2/100
1852/1852 [==============================] - 20s 11ms/step - loss: 0.8907 - accuracy: 0.8501 - val_loss: 0.6751 - val_accuracy: 0.8567
Epoch 3/100
1852/1852 [==============================] - 20s 11ms/step - loss: 0.5859 - accuracy: 0.8508 - val_loss: 0.5294 - val_accuracy: 0.8493
Epoch 4/100
1852/1852 [==============================] - 20s 11ms/step - loss: 0.4681 - accuracy: 0.8497 - val_loss: 0.5172 - val_accuracy: 0.7994
Epoch 5/100
1852/1852 [==============================] - 20s 11ms/step - loss: 0.4128 - accuracy: 0.8528 - val_loss: 0.4042 - val_accuracy: 0.8536
Epoch 6/100
1852/1852 [==============================] - 20s 11ms/step - loss: 0.3930 - accuracy: 0.8534 - val_loss: 0.3810 - val_accuracy: 0.8545
Epoch 7/100
1852/1852 [==============================] - 20s 11ms/step - loss: 0.3905 - accuracy: 0.8520 - val_loss

In [9]:
print("Training Accuracy over epochs:")
print(train_accuracy)

Training Accuracy over epochs:
[0.8268435597419739, 0.8501189947128296, 0.8508110046386719, 0.849713921546936, 0.8527688980102539, 0.8533934354782104, 0.8519924879074097, 0.8529376983642578, 0.8543217182159424, 0.8559083342552185, 0.853157103061676, 0.85435551404953, 0.8523300886154175, 0.8529039621353149, 0.8531233668327332, 0.8504565358161926, 0.8534947037696838, 0.8526001572608948, 0.853376567363739, 0.8531739711761475, 0.8539335131645203, 0.8511486053466797, 0.8523469567298889, 0.8525663614273071, 0.853241503238678, 0.8523806929588318, 0.8517393469810486, 0.8528532981872559, 0.8524988889694214, 0.8525326251983643, 0.8514186143875122, 0.8530896306037903, 0.8498827219009399, 0.8488362431526184, 0.8524650931358337, 0.8509798049926758, 0.8532583713531494, 0.8522962927818298, 0.8530896306037903, 0.851114809513092, 0.8515030145645142, 0.8515536785125732, 0.8515874147415161, 0.8531739711761475, 0.8519418835639954, 0.8527014255523682, 0.8528364300727844, 0.851621150970459, 0.85231322050094

In [10]:
print("\nValidation Accuracy over epochs:")
print(val_accuracy)


Validation Accuracy over epochs:
[0.8249392509460449, 0.8567377924919128, 0.849311351776123, 0.7994194030761719, 0.8536322116851807, 0.8545098304748535, 0.8555225729942322, 0.847488522529602, 0.8548474311828613, 0.8445179462432861, 0.8460707664489746, 0.8202133178710938, 0.8289225101470947, 0.8574129343032837, 0.8428976535797119, 0.8335133790969849, 0.8561301827430725, 0.8503240346908569, 0.849311351776123, 0.857885479927063, 0.8349311351776123, 0.8545098304748535, 0.8534296751022339, 0.8556575775146484, 0.8350661396980286, 0.854239821434021, 0.8334458470344543, 0.841749906539917, 0.8518768548965454, 0.8330407738685608, 0.8525519967079163, 0.85410475730896, 0.8494464159011841, 0.8506616353988647, 0.8570078611373901, 0.850796639919281, 0.8514717817306519, 0.8462733030319214, 0.8577504754066467, 0.8476235270500183, 0.8356737494468689, 0.845193088054657, 0.852619469165802, 0.8553875088691711, 0.8408047556877136, 0.8501214981079102, 0.8406022191047668, 0.8473535180091858, 0.83891439437866

In [ ]:
pip install altair vega_datasets


In [28]:
import altair as alt
import pandas as pd

# Sample data for demonstration
epochs = list(range(1, 101))  # 100 epochs
train_accuracy = [0.6 + i * 0.003 for i in range(100)]  # Dummy data
val_accuracy = [0.55 + i * 0.0035 for i in range(100)]  # Dummy data

# Create a DataFrame
df = pd.DataFrame({
    'Epoch': epochs,
    'Training Accuracy': train_accuracy,
    'Validation Accuracy': val_accuracy
})

# Melt the DataFrame for Altair
df_melted = df.melt('Epoch', var_name='Type', value_name='Accuracy')

# Create the plot
chart = alt.Chart(df_melted).mark_line(point=True).encode(
    x='Epoch',
    y='Accuracy',
    color='Type'
).properties(
    title='Training and Validation Accuracy over Epochs',
    width=800,
    height=400
).interactive()

chart.display()


alt.Chart(...)

In [30]:
last_epoch_df = pd.DataFrame({
    'Type': ['Training Accuracy', 'Validation Accuracy'],
    'Accuracy': [train_accuracy[-1], val_accuracy[-1]]
})

# Display the table
print("\nAccuracy in the Last Epoch:")
print(last_epoch_df)


Accuracy in the Last Epoch:
                  Type  Accuracy
0    Training Accuracy    0.8970
1  Validation Accuracy    0.8965


**Evaluation Metrics**

Save Model

In [14]:
model.save('trained_model')

INFO:tensorflow:Assets written to: trained_model\assets


INFO:tensorflow:Assets written to: trained_model\assets


Loading the Saved Model

In [33]:
loaded_model = tf.keras.models.load_model('trained_model')

**Recording Real-time Audio**

In [34]:
def record_audio():
    CHUNK = 1024
    FORMAT = pyaudio.paInt16
    CHANNELS = 1
    RATE = 44100
    RECORD_SECONDS = 5
    WAVE_OUTPUT_FILENAME = "output.wav"

    audio = pyaudio.PyAudio()

    stream = audio.open(format=FORMAT, channels=CHANNELS,
                        rate=RATE, input=True,
                        frames_per_buffer=CHUNK,
                        input_device_index=3)

    print("Recording...")

    frames = []

    for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
        data = stream.read(CHUNK)
        frames.append(data)

    print("Finished recording.")

    stream.stop_stream()
    stream.close()
    audio.terminate()

    wf = wave.open(WAVE_OUTPUT_FILENAME, 'wb')
    wf.setnchannels(CHANNELS)
    wf.setsampwidth(audio.get_sample_size(FORMAT))
    wf.setframerate(RATE)
    wf.writeframes(b''.join(frames))
    wf.close()

    return WAVE_OUTPUT_FILENAME



**Prediction on Audio Input**

In [35]:
audio_file = record_audio()

wf, sr = librosa.load(audio_file)
mfcc_wf = librosa.feature.mfcc(y=wf, sr=sr)
b = tf.keras.utils.pad_sequences(mfcc_wf, padding='post', maxlen=200)

prediction = loaded_model.predict(np.array([b]))

if prediction[0][0] > 0.5:
    print("Predicted gender: Male")
else:
    print("Predicted gender: Female")

Recording...
Finished recording.
1/1 [==============================] - 0s 99ms/step
Predicted gender: Male
